# 1. Importing the Libraries

In [1]:
# importing the libraries 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np

# 2. Data 

## * Loading the data

In [4]:
# loading the data 
df = pd.read_csv("DATA\lab2.csv")
# viewing the head 
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21.0
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98.0
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15.0
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,Oct-49,Pvt Ltd,1,39.0
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72.0


###  * Checking the data types and shape

In [5]:
# checking the type of each column in df
print(df.dtypes)

# lets check the length of dataset 
print(df.shape)

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours            float64
dtype: object
(2129, 13)


# 3. Handling Missing Values and labels 

In [6]:
# finding the total number of null values in the df 
df.isnull().sum()

enrollee_id                 0
city                        0
city_development_index      3
gender                    508
relevent_experience        20
enrolled_university        41
education_level            63
major_discipline          312
experience                  5
company_size              622
company_type              634
last_new_job               40
training_hours             20
dtype: int64

###  * city_development_index

In [7]:
# getting values 
ct_73 = df['city_development_index'][df['city' ]== 'city_73'].mode()[0]    # value for city_73
ct_45 = df['city_development_index'][df['city' ]== 'city_45'].mode()[0]    # value for city_45
ct_114 = df['city_development_index'][df['city' ]== 'city_114'].mode()[0]    # value for city_114

 
df['city_development_index'][30] = ct_73
df['city_development_index'][40] = ct_45
df['city_development_index'][92] = ct_114

###  * relevent_experience, education_level,major_discipline, enrolled_university, gender 

In [8]:

# we can see that the relevant experience value count is suffienctly large so lets the fill the missing
df.relevent_experience.fillna('Has relevent experience',inplace=  True)

# replacing missing education level with mode ( 63 values ) 
df.education_level.fillna(df.education_level.mode()[0],inplace = True)


# replacign missing values in major discipline using mode 
df.major_discipline.fillna(df.major_discipline.mode()[0], inplace = True)

# replacing missing values in enrolled university feature using mode
df.enrolled_university.fillna(df.enrolled_university.mode()[0],inplace = True)

# replacing nan values in gender with missing 
# df.gender.fillna('Missing',inplace= True)

###  * company_size feature

In [9]:
# checking the value counts
print(df.company_size.value_counts())

50-99        338
100-500      318
10000+       217
Oct-49       172
<10          163
1000-4999    143
500-999       88
5000-9999     68
Name: company_size, dtype: int64


In [10]:
# getting the first 312 index numbers of the rows for which the company_size is nan ( half of the total missing value for company_size)
first312 = df[df['company_size'].isnull()].index.tolist()[0:312]
# getting the rest index values 
restd = df[df['company_size'].isnull()].index.tolist()[312:]

df.loc[first312,'company_size'] = 4
df.loc[restd,'company_size'] = 3

In [11]:
# converting company size to numericals - 
def convert_cmp_size(val_string):
    if val_string == '<10':
        return 1
    if val_string == "Oct-49":
        return 2
    if val_string == '50-99':
        return 3
    if val_string == '100-500':
        return 4
    if val_string == '500-999':
        return 5
    if val_string == '1000-4999':
        return 6
    if val_string == "5000-9999":
        return 7
    if val_string == "10000+":
        return 8
    else:
        return val_string
df['company_size'] = df['company_size'].apply(convert_cmp_size)


# changing the datatype to integer ( ordered size)
df.company_size = df.company_size.astype(int)

### * experience feature 

In [12]:
# replacing missing experience with mode ( 5 values)
df.experience.fillna(df.experience.mode()[0],inplace= True)

# creating a function to handle the special values in experience column 
def experience_imputer(data):
    if data  == '<1':
        return 0
    if data == '>20':
        return 21
    else:
        return data 

# applying the function into experience column feature 
df.experience = df.experience.apply(experience_imputer)

# converting the object data type to numerical 
df.experience = df.experience.astype(int)


### * company_type

In [13]:
# filling company_type missing values by unkn_type 
df.company_type.fillna('Unkn_type', inplace = True)

In [14]:
df.company_type.value_counts()

Pvt Ltd                1141
Unkn_type               634
Public Sector           127
Funded Startup           97
Early Stage Startup      65
NGO                      53
Other                    12
Name: company_type, dtype: int64

In [15]:
# getting the index values of first 90% rows for which the company type is nan ( unknown)
f500 = df[df.company_type == 'Unkn_type'].index.tolist()[0:500]

# getting the rest index values 
r500 = df[df.company_type == 'Unkn_type'].index.tolist()[500:]


df.loc[f500,'company_type'] = 'Pvt Ltd'
df.loc[r500,'company_type'] = 'Public Sector'

In [16]:
# checking the value counts again 
df.company_type.value_counts()

Pvt Ltd                1641
Public Sector           261
Funded Startup           97
Early Stage Startup      65
NGO                      53
Other                    12
Name: company_type, dtype: int64

###     * last_new_job feature 

In [17]:

# replacing missing last_new_job values with mode ( 40 values )
df.last_new_job.fillna(df.last_new_job.mode()[0],inplace=True)


# converting last_new_job feature into numerical 
def last_job(data):
    if data == ">4":
        return 5
    if data == 'never':
        return 0
    else:
        return data
df.last_new_job = df.last_new_job.apply(last_job)
df.last_new_job = df.last_new_job.astype(int)

# 4. Handling outiers

In [18]:
# finding the median training hours for males - we interested in median bcs training_hours column has a lot of outliers
print('median_trainin_hours for male = ',df[df['gender']=='Male']['training_hours'].median())
print('median_trainin_hours for female = ',df[df['gender']=='Female']['training_hours'].median())

df.training_hours.fillna(45,inplace = True)

# fixing the outliers in training_hours
def fix_out(data):
    if data> 140:
        return df.training_hours.median()
    else:
        return data
        
# applying the function 
df.training_hours = df.training_hours.apply(fix_out)

median_trainin_hours for male =  48.0
median_trainin_hours for female =  42.0


# 5. Droping unecessary features 

In [19]:

# droping id column 
df.drop('enrollee_id',axis = 1,inplace = True)
# droping city column - unecessary 
df.drop('city', inplace= True, axis = 1)

### Filling gender data

In [20]:
df.gender.value_counts()

Male      1460
Female     137
Other       24
Name: gender, dtype: int64

In [21]:
df.gender.value_counts()
per_male= df.gender.value_counts()[0]/(len(df)-508)
per_female = df.gender.value_counts()[1]/(len(df) -508)
per_others = df.gender.value_counts()[2]/(len(df) -508)

# printing the percentages of each categories count 
print('per_male', per_male)
print('per_female', per_female)
print('per_others', per_others)


per_male 0.9006785934608267
per_female 0.08451573103022826
per_others 0.014805675508945095


In [22]:
df.gender.fillna('Missing', inplace = True)


g450 = df[df.gender == 'Missing'].index.tolist()[0:450]
g2450 = df[df.gender == 'Missing'].index.tolist()[450:500]
r450 = df[df.gender == 'Missing'].index.tolist()[500:]



df.loc[g450,'gender'] = 'Male'                # 90 percent of the missing replaced with male 
df.loc[g2450,'gender'] = 'Female'             # 8 percent of the missing  replaced with female
df.loc[r450,'gender'] = 'Other'               

In [23]:
df.gender.value_counts()

Male      1910
Female     187
Other       32
Name: gender, dtype: int64

### Checking whether there is any missing values in the data set 

In [24]:
df.gender.value_counts()

Male      1910
Female     187
Other       32
Name: gender, dtype: int64

In [25]:
df.isna().sum()

city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
dtype: int64

# 6. Dummy Encoding for categorical variables 

In [26]:

cat_features = df.drop(['city_development_index','experience','company_size','last_new_job','training_hours'] , axis = 1).columns

enc_df = pd.get_dummies(df[cat_features])

In [27]:
enc_df.head()

,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,education_level_High School,...,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd
0,0,1,0,1,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
1,1,0,0,1,0,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,1
2,0,1,0,0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,1
3,0,1,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0,1,0,1,0,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,1


In [29]:
enc_df.columns

Index(['gender_Female', 'gender_Male', 'gender_Other',
       'relevent_experience_Has relevent experience',
       'relevent_experience_No relevent experience',
       'enrolled_university_Full time course',
       'enrolled_university_Part time course',
       'enrolled_university_no_enrollment', 'education_level_Graduate',
       'education_level_High School', 'education_level_Masters',
       'education_level_Phd', 'education_level_Primary School',
       'major_discipline_Arts', 'major_discipline_Business Degree',
       'major_discipline_Humanities', 'major_discipline_No Major',
       'major_discipline_Other', 'major_discipline_STEM',
       'company_type_Early Stage Startup', 'company_type_Funded Startup',
       'company_type_NGO', 'company_type_Other', 'company_type_Public Sector',
       'company_type_Pvt Ltd'],
      dtype='object')

In [30]:

enc_final = enc_df.drop(['gender_Other','relevent_experience_No relevent experience','enrolled_university_no_enrollment','education_level_Phd','major_discipline_Other', 'company_type_Other'],axis =1)

In [31]:
# removing the coded features from our original dataframe 

df_num = df.drop(df[cat_features],axis = 1)
df_num.head()

,city_development_index,experience,company_size,last_new_job,training_hours
0,0.827,9,1,1,21.0
1,0.920,5,4,1,98.0
2,0.624,0,4,0,15.0
3,0.827,11,2,1,39.0
4,0.920,21,8,5,72.0


In [32]:
# merging the df_num and enc_final
final_df = pd.concat([df_num,enc_final],axis = 1)

In [33]:
# checking the dtypes 
print('total_no_features ' , len(final_df.dtypes))
print('---------------------------')
print(final_df.dtypes)

total_no_features  24
---------------------------
city_development_index                         float64
experience                                       int32
company_size                                     int32
last_new_job                                     int32
training_hours                                 float64
gender_Female                                    uint8
gender_Male                                      uint8
relevent_experience_Has relevent experience      uint8
enrolled_university_Full time course             uint8
enrolled_university_Part time course             uint8
education_level_Graduate                         uint8
education_level_High School                      uint8
education_level_Masters                          uint8
education_level_Primary School                   uint8
major_discipline_Arts                            uint8
major_discipline_Business Degree                 uint8
major_discipline_Humanities                      uint8
major_disciplin

# 7. Final DataFrame

In [34]:
final_df.head(10)

,city_development_index,experience,company_size,last_new_job,training_hours,gender_Female,gender_Male,relevent_experience_Has relevent experience,enrolled_university_Full time course,enrolled_university_Part time course,...,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_STEM,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Public Sector,company_type_Pvt Ltd
0,0.827,9,1,1,21.0,0,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
1,0.920,5,4,1,98.0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.624,0,4,0,15.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.827,11,2,1,39.0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,1
4,0.920,21,8,5,72.0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,1
5,0.899,10,4,2,12.0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1
6,0.624,0,4,1,11.0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,1
7,0.920,21,4,5,81.0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
8,0.878,14,4,4,2.0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,1
9,0.624,3,3,1,4.0,0,1,1,1,0,...,0,0,0,0,1,0,1,0,0,0
